In [2]:
%cd ..

/home/anthonypark6904/CLR-module


In [3]:
import flax 
import flax.linen as nn

from flax import jax_utils
from flax.training import train_state, common_utils, checkpoints
from flax.core import freeze, unfreeze

import os

os.environ["TPU_CHIPS_PER_HOST_BOUNDS"] = "1,1,1"
os.environ["TPU_HOST_BOUNDS"] = "1,1,1"
# Different per process:
os.environ["TPU_VISIBLE_DEVICES"] = "2" 
# Pick a unique port per process
os.environ["TPU_MESH_CONTROLLER_ADDRESS"] = "localhost:8476"
os.environ["TPU_MESH_CONTROLLER_PORT"] = "8476"

2022-09-21 06:08:26.543890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-09-21 06:08:26.578144: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-21 06:08:27.511974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-09-21 06:08:27.512166: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


In [4]:
from trainer import *


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
class SimpleNet(nn.Module):
    @nn.compact
    def __call__(self, x):
        metircs = {}
        x = nn.normalization.BatchNorm(False)(x)
        metrics['BatchNorm_1'] = x
        x = nn.Dense(5)(x)
        metrics['Dense'] = x
        
        x = nn.normalization.BatchNorm(True)(x)
        metrics['BatchNorm_2'] = x
        
        return x, metrics

In [43]:
import jax
import jax.numpy as jnp

In [44]:
key = jax.random.PRNGKey(30)
params = SimpleNet().init(key, jnp.ones((5,10)))

In [45]:
params

FrozenDict({
    batch_stats: {
        BatchNorm_0: {
            mean: DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
            var: DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
        },
        BatchNorm_1: {
            mean: DeviceArray([0., 0., 0., 0., 0.], dtype=float32),
            var: DeviceArray([1., 1., 1., 1., 1.], dtype=float32),
        },
    },
    params: {
        BatchNorm_0: {
            scale: DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
            bias: DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
        },
        Dense_0: {
            kernel: DeviceArray([[ 0.4726773 ,  0.6936623 ,  0.6897993 , -0.19865191,
                           0.22411938],
                         [ 0.45619652,  0.06175263,  0.23109236, -0.06140082,
                          -0.17739616],
                         [-0.6689891 ,  0.21679907,  0.31855845, -0.19363695,
                 

In [46]:
_, metrics = SimpleNet().apply(params, jax.random.normal(key, (5,10)))

ModifyScopeVariableError: Cannot update variable "mean" in "/BatchNorm_0" because collection "batch_stats" is immutable. (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.ModifyScopeVariableError)

In [47]:
metrics

{'BatchNorm_2': DeviceArray([[0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.]], dtype=float32),
 'BatchNorm_1': DeviceArray([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 'Dense': DeviceArray([[0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0.]], dtype=float32)}

In [49]:
jnp.mean(jnp.ones((16, 10, 20, 512)), axis=[1,2]).shape

(16, 512)